<a href="https://colab.research.google.com/github/IzaakGagnon/Integrated_Information_Testing/blob/main/Computing_The_Total_Number_Of_Partitions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!python -m pip install -U git+https://github.com/wmayner/pyphi.git@feature/iit-4.0
import pyphi
pyphi.config.PROGRESS_BARS = False
pyphi.config.PARALLEL = False
pyphi.config.SHORTCIRCUIT_SIA = False
pyphi.config.WELCOME_OFF = True
pyphi.config.REPERTOIRE_DISTANCE = "GENERALIZED_INTRINSIC_DIFFERENCE"

  Cloning https://github.com/wmayner/pyphi.git (to revision feature/iit-4.0) to /tmp/pip-req-build-j3k4cc2h
  Running command git clone --filter=blob:none --quiet https://github.com/wmayner/pyphi.git /tmp/pip-req-build-j3k4cc2h
  Resolved https://github.com/wmayner/pyphi.git to commit 6b83cbdbbcdca75289415fe096adbac5f2ec7a4d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 662.6/662.6 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 MB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.0/252.0 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.4/201.4 kB 19.1 MB/s eta 0:00:00
     ━━━━━━

In [2]:
import pyphi
import time
import functools
import itertools
from itertools import chain, product
import numpy as np
from more_itertools import distinct_permutations
from toolz import unique
from pyphi import combinatorics
from pyphi.cache import cache
from pyphi.conf import config, fallback
from pyphi.direction import Direction
from pyphi.models.cuts import (
    Bipartition,
    CompleteGeneralKCut,
    CompleteGeneralSetPartition,
    Cut,
    GeneralKCut,
    GeneralSetPartition,
    KPartition,
    Part,
    SystemPartition,
    Tripartition,
)
from pyphi.registry import Registry
from pyphi.partition import system_partition_types
def normalize_rows(matrix):
    ### Scales probabilities in a matrix so that they satisfy the markov property. Proof that CI still holds not completed yet
    num_rows = matrix.shape[0]
    normalized_matrix = np.zeros_like(matrix)  # Create an empty matrix of the same shape
    for i in range(num_rows):
        row = matrix[i, :]
        current_sum = np.sum(row)
        if current_sum > 0:
            scaling_factor = 1.0 / current_sum
            normalized_row = row * scaling_factor
            normalized_matrix[i, :] = normalized_row
        else: print("Zero_Sum_Error: Problem With Values Generated")
    return normalized_matrix
def create_noisy_network(size):
    ### Obtains a random network of nodes of a given size, directly ready for computing Phi.
    return pyphi.Network(normalize_rows(pyphi.convert.state_by_node2state_by_state(np.random.rand(2**size,size))))
def create_noiseless_network(size):
  return pyphi.Network(np.random.randint(0,2,(2**size,size)))
partitions_list = []
random_proportion = 0.022
### The following is altered code from "SET_UNI/BI" with a few changes.
def _unidirectional_set_partitions(node_indices,random_proportion, node_labels=None):
    """Generate all unidirectional set partitions of a set of nodes."""                   ### Generate a partition one at a time "Hence being a generator"
    if len(node_indices) == 1 or config.SYSTEM_PARTITION_INCLUDE_COMPLETE:
        yield CompleteGeneralSetPartition(node_indices, node_labels=node_labels)
    _node_indices = set(range(len(node_indices)))

    for partition in combinatorics.set_partitions(_node_indices, nontrivial=True):
      partitions_list.append(partition)  ;                        ### One change here: what chance do we consider such partition
      for directions in product(Direction.all(), repeat=len(partition)):                ### Go ahead and consider it.
            cut_matrix = np.zeros([len(_node_indices), len(_node_indices)], dtype=int)
            for part, direction in zip(partition, directions):
                nonpart = list(_node_indices - set(part))
                if direction == Direction.CAUSE:
                    source, target = nonpart, part
                else:
                    source, target = part, nonpart
                cut_matrix[np.ix_(source, target)] = 1
                if direction == Direction.BIDIRECTIONAL:
                    cut_matrix[np.ix_(target, source)] = 1
            yield GeneralSetPartition(
                node_indices,
                cut_matrix,
                node_labels=node_labels,
                set_partition=partition,
            )
@system_partition_types.register("SET_UNI/BI_Partition_Counter")
@functools.wraps(_unidirectional_set_partitions)
def unidirectional_set_partitions(node_indices, node_labels=None):
    yield from unique(
        _unidirectional_set_partitions(node_indices,random_proportion, node_labels=node_labels))

In [3]:
bipartitions_list = []
@cache(cache={}, maxmem=None)
def bipartition_indices(N):
    """Return indices for undirected bipartitions of a sequence.

    Args:
        N (int): The length of the sequence.

    Returns:
        list: A list of tuples containing the indices for each of the two
        parts.

    Example:
        >>> N = 3
        >>> bipartition_indices(N)
        [((), (0, 1, 2)), ((0,), (1, 2)), ((1,), (0, 2)), ((0, 1), (2,))]
    """
    result = []
    if N <= 0:
        return result

    for i in range(2 ** (N - 1)):
        part = [[], []]
        for n in range(N):
            bit = (i >> n) & 1
            part[bit].append(n)
        result.append((tuple(part[1]), tuple(part[0])))
    return result

def bipartition(seq, nontrivial=False):
    """Return a list of bipartitions for a sequence.

    Args:
        a (Iterable): The sequence to partition.

    Returns:
        list[tuple[tuple]]: A list of tuples containing each of the two
        partitions.

    Example:
        >>> bipartition((1,2,3))
        [((), (1, 2, 3)), ((1,), (2, 3)), ((2,), (1, 3)), ((1, 2), (3,))]
    """
    bipartitions = [
        (tuple(seq[i] for i in part0_idx), tuple(seq[j] for j in part1_idx))
        for part0_idx, part1_idx in bipartition_indices(len(seq))
    ]
    if nontrivial:
        return bipartitions[1:]
    return bipartitions


@cache(cache={}, maxmem=None)
def directed_bipartition_indices(N):
    """Return indices for directed bipartitions of a sequence.

    Args:
        N (int): The length of the sequence.

    Returns:
        list: A list of tuples containing the indices for each of the two
        parts.

    Example:
        >>> N = 3
        >>> directed_bipartition_indices(N)  # doctest: +NORMALIZE_WHITESPACE
        [((), (0, 1, 2)),
         ((0,), (1, 2)),
         ((1,), (0, 2)),
         ((0, 1), (2,)),
         ((2,), (0, 1)),
         ((0, 2), (1,)),
         ((1, 2), (0,)),
         ((0, 1, 2), ())]
    """
    indices = bipartition_indices(N)
    return indices + [idx[::-1] for idx in indices[::-1]]


def _bipartitions_to_temporal_system_partitions(func):
    """Decorator to return temporally-directed SystemPartition objects from a
    set of bipartitions.
    """
    @functools.wraps(func)
    def wrapper(*args, node_labels=None, **kwargs):
        for bipartition in func(*args, **kwargs):
            for direction in Direction.both():
                bipartitions_list.append([bipartition,direction])
                yield SystemPartition(
                    direction,
                    bipartition[0],
                    bipartition[1],
                    node_labels=node_labels,
                )

    return wrapper

@system_partition_types.register("TEMPORAL_DIRECTED_BI_Partition_Counter")
@_bipartitions_to_temporal_system_partitions
def system_temporal_directed_bipartitions(nodes):
    # Don't consider trivial partitions where one part is empty
    return directed_bipartition_counter(nodes, nontrivial=True)

def directed_bipartition_counter(seq, nontrivial=False):
    """Return a list of directed bipartitions for a sequence.

    Args:
        seq (Iterable): The sequence to partition.

    Returns:
        list[tuple[tuple]]: A list of tuples containing each of the two
        parts.

    Example:
        >>> directed_bipartition((1, 2, 3))  # doctest: +NORMALIZE_WHITESPACE
        [((), (1, 2, 3)),
         ((1,), (2, 3)),
         ((2,), (1, 3)),
         ((1, 2), (3,)),
         ((3,), (1, 2)),
         ((1, 3), (2,)),
         ((2, 3), (1,)),
         ((1, 2, 3), ())]
    """
    bipartitions = [
        (tuple(seq[i] for i in part0_idx), tuple(seq[j] for j in part1_idx))
        for part0_idx, part1_idx in directed_bipartition_indices(len(seq))
    ]
    if nontrivial:
        # The first and last partitions have a part that is empty; skip them.
        # NOTE: This depends on the implementation of
        # `directed_partition_indices`.
        return bipartitions[1:-1]

    return bipartitions


In [7]:
np.random.seed(99999)
pyphi.config.SYSTEM_PARTITION_TYPE = "TEMPORAL_DIRECTED_BI_Partition_Counter"
bipartitions_list = []
network = create_noisy_network(3)
state = [np.random.choice([0, 1]) for i in range(3)]
subsystem_cause = pyphi.Subsystem(network,state,backward_tpm=True)
subsystem_effect = pyphi.Subsystem(network,state,backward_tpm=False)
sia = pyphi.backwards.sia(subsystem_cause,subsystem_effect)
print(sia)
print(bipartitions_list)
print(len(bipartitions_list)+1)

np.random.seed(0)
pyphi.config.SYSTEM_PARTITION_TYPE = "SET_UNI/BI_Partition_Counter"
partitions_list = []
network = create_noisy_network(3)
state = [np.random.choice([0, 1]) for i in range(3)]
subsystem_cause = pyphi.Subsystem(network,state,backward_tpm=True)
subsystem_effect = pyphi.Subsystem(network,state,backward_tpm=False)
sia = pyphi.backwards.sia(subsystem_cause,subsystem_effect)
print(sia)
print(partitions_list)
print(len(partitions_list)+1) ### Add one for the trival case of the empty set | System

┌───────────────────────────────────────────────────────┐
│     SystemIrreducibilityAnalysis                      │
│ ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━                  │
│      Subsystem:   n0,n1,n2                            │
│  Current state:   (0,1,0)                             │
│            φ_s: -0.15048677590817056                  │
│ Normalized φ_s: -0.07524338795408528                  │
│          CAUSE:   (0,0,1)                             │
│           II_c:  0.2751160568687054                   │
│         EFFECT:   (1,0,1)                             │
│           II_e:  0.6104483910499294                   │
│   #(tied MIPs):  0                                    │
│      Partition:                                       │
│                 SystemPartition [n2] ━━/ /━━▶ [n0,n1] │
└───────────────────────────────────────────────────────┘
[[((0,), (1, 2)), CAUSE], [((0,), (1, 2)), EFFECT], [((1,), (0, 2)), CAUSE], [((1,), (0, 2)), EFFECT], [((0, 1), (2,)), CAUSE], [(

In [16]:
def factorial(num):
    if num == 0 or num == 1:
        return 1
    result = 1
    for i in range(2, num + 1):
        result *= i
    return result
def binomial_coefficient(n, k):
    if k < 0 or k > n:
        return 0
    return factorial(n) / (factorial(k) * factorial(n - k))
def stirling_2(n,k):
  count = 0
  for i in range(k+1):
    count = count + (-1)**i * binomial_coefficient(k, i) * (k - i)**n
  return count / factorial(k)
def total_partitions(n, max_partition_count):
  count = 0
  for i in range(max_partition_count+1):
    count = count + stirling_2(n, i)
  return count
print(factorial(4))
print(binomial_coefficient(4,2))
print(total_partitions(4,4))

24
6.0
15.0
